# MIT (TA1): From Paper and Code to annotated extraction


## 0. Preprocessing

In [1]:
import ast, json, requests, os
GPT_KEY = os.environ.get('OPENAI_API_KEY')
API_ROOT = "http://localhost:8000/"

## 1. MIT annotation end to end pipeline

#### We bring every annotation step together: for the original paper text, let's integrate all the extraction modules and output the MIT extraction:
[http://3.83.68.208/#/Paper-2-annotated-vars/upload_file_annotate_annotation_upload_file_extract__post](http://3.83.68.208/#/Paper-2-annotated-vars/upload_file_annotate_annotation_upload_file_extract__post)

In [2]:
scenario_1_paper = "text_ijerph-18-09027.txt"
with open(scenario_1_paper, 'rb') as f:
    files = {'file': ('filename', f)}
    params = {"gpt_key": GPT_KEY}
    response = requests.post(API_ROOT + "annotation/upload_file_extract",  params=params,  files=files)
    assert response.status_code == 200, response.text
json_str = response.text

In [3]:
import ta1viewer # SAME REPO AS EXTRACTION VIEWER
from ta1viewer import display_json
#pip install git+https://github.com/orm011/ta1viewer.git@main
display_json(json_str)

In [4]:
import json

json_object = json.dumps(json.loads(json_str), indent=4)
with open("m_temp_scenario_1.json", "w") as f:
    f.write(json_object)

## 2. Interacting with the University of Arizona extraction

#### With both the University of Arizona and MIT extractions, we first build the entity matching mapping for all the extracted variable entities, and then integrate the two integration together with the unified TA1 data model.
[http://3.83.68.208/#/TA1-Integration/upload_files_integration_integration_get_mapping_post](http://3.83.68.208/#/TA1-Integration/upload_files_integration_integration_get_mapping_post)

In [5]:
with open('m_temp_scenario_1.json', 'rb') as f_mit, open('ijerph-18-09027.json', 'rb') as f_arizona:
    files = { 'mit_file': ('bucky__mit-extraction_id.json', f_mit, 'application/json'),
        'arizona_file': ('bucky_arizona_output_example.json', f_arizona, 'application/json')}
    params = {"gpt_key": GPT_KEY}
    response = requests.post(API_ROOT + "integration/get_mapping", params=params, files=files)
    assert response.status_code == 200, response.text
json_str = response.text

In [6]:
display_json(json_str)